In [1]:
## Dependencies

import pandas as pd
import pathlib as path
import numpy as np
import matplotlib as plt
import hvplot.pandas
import requests
import json
from pprint import pprint


## CREATE DATA FRAME FOR DEATHS STATISTICS
    ## Daily data from 2020 to 2024
    # Resource : CSV file from World Health Organization 
        # https://data.who.int/dashboards/covid19/data?n=o

In [2]:
## create DataFrame death statistics

### import and read csv file

path_death = 'Resources/WHO-COVID-19-global-daily-data.csv'
death_df = pd.read_csv(path_death)

# death_df exploration

print(death_df.shape)
print(death_df.dtypes)
print(death_df.head())

(413760, 8)
Date_reported         object
Country_code          object
Country               object
WHO_region            object
New_cases            float64
Cumulative_cases       int64
New_deaths           float64
Cumulative_deaths      int64
dtype: object
  Date_reported Country_code     Country WHO_region  New_cases  \
0    2020-01-04           AI    Anguilla        AMR        NaN   
1    2020-01-04           AZ  Azerbaijan        EUR        NaN   
2    2020-01-04           BB    Barbados        AMR        NaN   
3    2020-01-04           BR      Brazil        AMR        NaN   
4    2020-01-04           BD  Bangladesh       SEAR        0.0   

   Cumulative_cases  New_deaths  Cumulative_deaths  
0                 0         NaN                  0  
1                 0         NaN                  0  
2                 0         NaN                  0  
3                 0         NaN                  0  
4                 0         0.0                  0  


In [3]:
### change data type for columns INT + date

death_df = death_df.astype({
    'Date_reported' : 'datetime64[ns]',
    'New_cases': "Float64", 
    'Cumulative_cases': "Float64", 
    'Cumulative_deaths' : 'Float64'
    })
death_df.dtypes

Date_reported        datetime64[ns]
Country_code                 object
Country                      object
WHO_region                   object
New_cases                   Float64
Cumulative_cases            Float64
New_deaths                  float64
Cumulative_deaths           Float64
dtype: object

In [4]:
### count of total cases and deaths by country GRoupBy

death_total_country = death_df.groupby('Country').agg({
    'Cumulative_cases' : 'max',
    'Cumulative_deaths' : 'max'
})
death_total_country
# death_total_country.sort_values(ascending=False)

,Cumulative_cases,Cumulative_deaths
Country,,
Afghanistan,235214.0,7998.0
Albania,335047.0,3605.0
Algeria,272170.0,6881.0
American Samoa,8359.0,34.0
Andorra,48015.0,159.0
...,...,...
Wallis and Futuna Is,3760.0,9.0
West Bank and Gaza Strip,703228.0,5708.0
Yemen,11945.0,2159.0


In [5]:
### add column total_cases + total_death for each country

death_df_clean = pd.merge(death_df,death_total_country,on='Country')

death_df_clean

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases_x,New_deaths,Cumulative_deaths_x,Cumulative_cases_y,Cumulative_deaths_y
0,2020-01-04,AI,Anguilla,AMR,<NA>,0.0,NaN,0.0,3904.0,12.0
1,2020-01-04,AZ,Azerbaijan,EUR,<NA>,0.0,NaN,0.0,836285.0,10353.0
2,2020-01-04,BB,Barbados,AMR,<NA>,0.0,NaN,0.0,108815.0,593.0
3,2020-01-04,BR,Brazil,AMR,<NA>,0.0,NaN,0.0,37511921.0,702116.0
4,2020-01-04,BD,Bangladesh,SEAR,0.0,0.0,0.0,0.0,2051417.0,29499.0
...,...,...,...,...,...,...,...,...,...,...
413755,2024-09-22,BB,Barbados,AMR,<NA>,108815.0,NaN,593.0,108815.0,593.0
413756,2024-09-22,BR,Brazil,AMR,<NA>,37511921.0,NaN,702116.0,37511921.0,702116.0
413757,2024-09-22,AZ,Azerbaijan,EUR,<NA>,836285.0,NaN,10353.0,836285.0,10353.0
413758,2024-09-22,CO,Colombia,AMR,<NA>,6394022.0,NaN,142727.0,6394022.0,142727.0


In [6]:
### rename new columns 

death_df_clean = death_df_clean.rename(columns={
    'Cumulative_deaths_x' : 'Cumulative_deaths',
    'Cumulative_deaths_y' : "Total_deaths",
    'Cumulative_cases_x' : 'Cumulative_cases',
    'Cumulative_cases_y' : "Total_cases"
})
death_df_clean

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Total_cases,Total_deaths
0,2020-01-04,AI,Anguilla,AMR,<NA>,0.0,NaN,0.0,3904.0,12.0
1,2020-01-04,AZ,Azerbaijan,EUR,<NA>,0.0,NaN,0.0,836285.0,10353.0
2,2020-01-04,BB,Barbados,AMR,<NA>,0.0,NaN,0.0,108815.0,593.0
3,2020-01-04,BR,Brazil,AMR,<NA>,0.0,NaN,0.0,37511921.0,702116.0
4,2020-01-04,BD,Bangladesh,SEAR,0.0,0.0,0.0,0.0,2051417.0,29499.0
...,...,...,...,...,...,...,...,...,...,...
413755,2024-09-22,BB,Barbados,AMR,<NA>,108815.0,NaN,593.0,108815.0,593.0
413756,2024-09-22,BR,Brazil,AMR,<NA>,37511921.0,NaN,702116.0,37511921.0,702116.0
413757,2024-09-22,AZ,Azerbaijan,EUR,<NA>,836285.0,NaN,10353.0,836285.0,10353.0
413758,2024-09-22,CO,Colombia,AMR,<NA>,6394022.0,NaN,142727.0,6394022.0,142727.0


# CREATE DATAFRAME FOR VACCINATION STATISTICS
    # Statistic end of period (static)
    # same source WHO :  https://data.who.int/dashboards/covid19/data?n=o

In [7]:
## create DataFrame Vaccination 

## Import and read csv file

path_Vaccination = 'Resources/vaccination-data.csv'
Vaccination_df = pd.read_csv(path_Vaccination)

# Vaccination _df exploration

print(Vaccination_df.shape)
print(Vaccination_df.dtypes)
print(Vaccination_df.head())

(215, 16)
COUNTRY                                  object
ISO3                                     object
WHO_REGION                               object
DATA_SOURCE                              object
DATE_UPDATED                             object
TOTAL_VACCINATIONS                      float64
PERSONS_VACCINATED_1PLUS_DOSE           float64
TOTAL_VACCINATIONS_PER100               float64
PERSONS_VACCINATED_1PLUS_DOSE_PER100    float64
PERSONS_LAST_DOSE                       float64
PERSONS_LAST_DOSE_PER100                float64
VACCINES_USED                           float64
FIRST_VACCINE_DATE                       object
NUMBER_VACCINES_TYPES_USED              float64
PERSONS_BOOSTER_ADD_DOSE                float64
PERSONS_BOOSTER_ADD_DOSE_PER100         float64
dtype: object
       COUNTRY ISO3 WHO_REGION DATA_SOURCE DATE_UPDATED  TOTAL_VACCINATIONS  \
0        Aruba  ABW       AMRO   REPORTING   2023-12-29            217124.0   
1  Afghanistan  AFG       EMRO   REPORTING   2023-

In [8]:
## change data type for columns INT + date

Vaccination_df = Vaccination_df.astype({
    'DATE_UPDATED' : 'datetime64[ns]',
    'FIRST_VACCINE_DATE' : 'datetime64[ns]'
    })
Vaccination_df.dtypes

COUNTRY                                         object
ISO3                                            object
WHO_REGION                                      object
DATA_SOURCE                                     object
DATE_UPDATED                            datetime64[ns]
TOTAL_VACCINATIONS                             float64
PERSONS_VACCINATED_1PLUS_DOSE                  float64
TOTAL_VACCINATIONS_PER100                      float64
PERSONS_VACCINATED_1PLUS_DOSE_PER100           float64
PERSONS_LAST_DOSE                              float64
PERSONS_LAST_DOSE_PER100                       float64
VACCINES_USED                                  float64
FIRST_VACCINE_DATE                      datetime64[ns]
NUMBER_VACCINES_TYPES_USED                     float64
PERSONS_BOOSTER_ADD_DOSE                       float64
PERSONS_BOOSTER_ADD_DOSE_PER100                float64
dtype: object

In [9]:
# print(death_df['Country_code'].unique())
# print(Vaccination_df['ISO3'].unique())

## Create 1 DataFrame : Deaths + Vaccination statistics
    ## Merge 2 previous DF

In [10]:
##  merge DF death + vaccination

death_vaccins_df = pd.merge(
    death_df_clean,
    Vaccination_df,
    how="inner",
    left_on = 'Country',
    right_on = 'COUNTRY'
    )

print(death_vaccins_df.head())


  Date_reported Country_code     Country WHO_region  New_cases  \
0    2020-01-04           AI    Anguilla        AMR       <NA>   
1    2020-01-04           AZ  Azerbaijan        EUR       <NA>   
2    2020-01-04           BB    Barbados        AMR       <NA>   
3    2020-01-04           BR      Brazil        AMR       <NA>   
4    2020-01-04           BD  Bangladesh       SEAR        0.0   

   Cumulative_cases  New_deaths  Cumulative_deaths  Total_cases  Total_deaths  \
0               0.0         NaN                0.0       3904.0          12.0   
1               0.0         NaN                0.0     836285.0       10353.0   
2               0.0         NaN                0.0     108815.0         593.0   
3               0.0         NaN                0.0   37511921.0      702116.0   
4               0.0         0.0                0.0    2051417.0       29499.0   

   ... PERSONS_VACCINATED_1PLUS_DOSE TOTAL_VACCINATIONS_PER100  \
0  ...                       10858.0              

In [11]:
# explore columns

print(death_vaccins_df.columns)

Index(['Date_reported', 'Country_code', 'Country', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths', 'Total_cases',
       'Total_deaths', 'COUNTRY', 'ISO3', 'WHO_REGION', 'DATA_SOURCE',
       'DATE_UPDATED', 'TOTAL_VACCINATIONS', 'PERSONS_VACCINATED_1PLUS_DOSE',
       'TOTAL_VACCINATIONS_PER100', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100',
       'PERSONS_LAST_DOSE', 'PERSONS_LAST_DOSE_PER100', 'VACCINES_USED',
       'FIRST_VACCINE_DATE', 'NUMBER_VACCINES_TYPES_USED',
       'PERSONS_BOOSTER_ADD_DOSE', 'PERSONS_BOOSTER_ADD_DOSE_PER100'],
      dtype='object')


In [12]:
# selected useful columns for death_vaccins_df 

death_vaccins_df_clean = death_vaccins_df[['Country','Country_code','WHO_region',
                                    'New_cases','Cumulative_cases', 'New_deaths', 'Cumulative_deaths',
                                    'Total_cases','Total_deaths',
                                    'DATE_UPDATED','TOTAL_VACCINATIONS','TOTAL_VACCINATIONS_PER100',
                                    'PERSONS_BOOSTER_ADD_DOSE_PER100'
                                    ]]

death_vaccins_df_clean.head()

,Country,Country_code,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Total_cases,Total_deaths,DATE_UPDATED,TOTAL_VACCINATIONS,TOTAL_VACCINATIONS_PER100,PERSONS_BOOSTER_ADD_DOSE_PER100
0,Anguilla,AI,AMR,<NA>,0.0,NaN,0.0,3904.0,12.0,2023-12-29,24864.0,166.0,22.0
1,Azerbaijan,AZ,EUR,<NA>,0.0,NaN,0.0,836285.0,10353.0,2023-12-23,13829360.0,136.0,35.0
2,Barbados,BB,AMR,<NA>,0.0,NaN,0.0,108815.0,593.0,2023-12-29,384049.0,134.0,22.0
3,Brazil,BR,AMR,<NA>,0.0,NaN,0.0,37511921.0,702116.0,2023-12-29,516237106.0,243.0,52.0
4,Bangladesh,BD,SEAR,0.0,0.0,0.0,0.0,2051417.0,29499.0,2023-10-18,362229859.0,220.0,42.0


In [13]:
### little check

print(death_vaccins_df_clean.shape)
print(death_vaccins_df_clean.columns)

(351696, 13)
Index(['Country', 'Country_code', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths', 'Total_cases',
       'Total_deaths', 'DATE_UPDATED', 'TOTAL_VACCINATIONS',
       'TOTAL_VACCINATIONS_PER100', 'PERSONS_BOOSTER_ADD_DOSE_PER100'],
      dtype='object')


In [ ]:
## Export to CSV


## Code to extract the official referential of Countries used by the World Bank
    # OUtput file CSV availabe in directory Resources
    # source API World Bank
        ### this code should / must be runned once (no need to update the file)
        ## will be used below to extract data (GDP + population)
        ## can be used for different other requests

In [14]:
## retrieve all the countries in API

### generate DF Countries

page = 1   ## there are 6 pages in the json file
data = []
url_countries = 'https://api.worldbank.org/v2/country?format=json'

for page in range(1,7):
    response = requests.get(f"{url_countries}&page={page}")
    json_data = response.json()
    
    # Add the data from the current page
    data.extend(json_data[1])  
    
# for i in data:
#     print(i['id'], i['name']) 


## create DF and select columns

countries_df = pd.DataFrame(data)
countries_df = countries_df[['id', 'iso2Code', 'name', 'region','capitalCity', 'longitude', 'latitude']]



In [15]:
### TO RUN ONLY ONCE
### Export to csv file

country_path = 'Resources/countries_who.csv'
countries_df.to_csv(country_path)

In [16]:
countries_df.columns

Index(['id', 'iso2Code', 'name', 'region', 'capitalCity', 'longitude',
       'latitude'],
      dtype='object')

# CREATE DataFrame WITH GDP & POPULATION DATA FOR EACH COUNTRY
    # From Worldbank API
    # 2 different Api used
    # generate 2 different DF which will be merged and cleaned
    # output csv file available in directory Resources 

In [17]:
print(f"start code")
print('---------------------------')

## create list of countries from previous DF

country_list = countries_df['id'].tolist()

gdp_list = []
pop_list = []

# indicator = ['NY.GDP.MKTP.CD','SP.POP.TOT']

# each country has its onwn end Point, so need to loop through all end points and extract needed data

for id in country_list :

    print(f"--------------------------")
    print(f"country {id}")

    ## collect data for GDP

    try:  

       

        link_gdp = f'https://api.worldbank.org/v2/country/{id}/indicator/NY.GDP.MKTP.CD?date=2020:2023&format=json'
        response_gdp = requests.get(link_gdp).json()

        # pprint(response_gdp)

        gdp_list.append({
            'country_ID' : response_gdp[1][0]['country']['id'],
            'country_name' : response_gdp[1][0]['country']['value'],
            'GDP_2023' : response_gdp[1][0]['value'] ,
            'GDP_2022' : response_gdp[1][1]['value'] ,
            'GDP_2021' : response_gdp[1][2]['value'] ,
            'GDP_2020' : response_gdp[1][3]['value']
            })
        

    except: 
        print("Country not found GDP. Skipping...")

    ## collect data for Population

    try:

        link_pop = f'https://api.worldbank.org/v2/country/{id}/indicator/SP.POP.TOTL?date=2020:2023&format=json'
        response_pop = requests.get(link_pop).json()

        # pprint(response_pop)

        pop_list.append({
            'country_ID' : response_gdp[1][0]['country']['id'],
            'country_name' : response_gdp[1][0]['country']['value'],              
            'pop_2023' : response_pop[1][0]['value'] ,
            'pop_2022' :response_pop[1][1]['value'] ,
            'pop_2021' :response_pop[1][2]['value'] ,
            'pop_2020' : response_pop[1][3]['value']
            })
    except:   
        print("Country not found POP. Skipping...")

start code
---------------------------
--------------------------
country ABW
--------------------------
country AFE
--------------------------
country AFG
--------------------------
country AFR
Country not found GDP. Skipping...
Country not found POP. Skipping...
--------------------------
country AFW
--------------------------
country AGO
--------------------------
country ALB
--------------------------
country AND
--------------------------
country ARB
--------------------------
country ARE
--------------------------
country ARG
--------------------------
country ARM
--------------------------
country ASM
--------------------------
country ATG
--------------------------
country AUS
--------------------------
country AUT
--------------------------
country AZE
--------------------------
country BDI
--------------------------
country BEA
Country not found GDP. Skipping...
Country not found POP. Skipping...
--------------------------
country BEC
Country not found GDP. Skipping...
Countr

In [18]:
## Create DF

gdp_df = pd.DataFrame(gdp_list)
pop_df = pd.DataFrame(pop_list)

## Merge to create gdp_pop_df

gdp_pop_df = pd.merge(gdp_df,pop_df,on='country_ID')

## Rename and select columns

gdp_pop_df = gdp_pop_df.drop(columns='country_name_y')
gdp_pop_df = gdp_pop_df.rename(columns={'country_name_x' : 'country_name'})

gdp_pop_df = gdp_pop_df.dropna(how='all')


## Export to CSV file 

gdp_pop_path = 'Resources/countries_gdp_pop.csv'
gdp_pop_df.to_csv(gdp_pop_path)

gdp_pop_df.head()

,country_ID,country_name,GDP_2023,GDP_2022,GDP_2021,GDP_2020,pop_2023,pop_2022,pop_2021,pop_2020
0,AW,Aruba,NaN,3.544708e+09,3.103184e+09,2.558906e+09,106277.0,106445.0,106537.0,106585.0
1,ZH,Africa Eastern and Southern,1.236163e+12,1.183962e+12,1.086772e+12,9.290741e+11,739108306.0,720859132.0,702977106.0,685112979.0
2,AF,Afghanistan,NaN,1.450216e+10,1.426650e+10,1.995593e+10,42239854.0,41128771.0,40099462.0,38972230.0
3,ZI,Africa Western and Central,7.965862e+11,8.771408e+11,8.459930e+11,7.871467e+11,502789511.0,490330870.0,478185907.0,466189102.0
4,AO,Angola,8.472296e+10,1.043997e+11,6.650513e+10,4.850156e+10,36684202.0,35588987.0,34503774.0,33428486.0


# Codes not to be used except to test the API's

In [ ]:
# link_gdp = f'https://api.worldbank.org/v2/country/ABW/indicator/NY.GDP.MKTP.CD?date=2020:2023&format=json'

# response_gdp = requests.get(link_gdp).json()

# # pprint(response_gdp)


# print(f'country ID ={response_gdp[1][0]['country']['id']}')
# print(f'country ID ={response_gdp[1][0]['country']['value']}')
# print(f'GDP_2023 ={response_gdp[1][0]['value']}')
# print(f'GDP_2022 ={response_gdp[1][1]['value']}')
# print(f'GDP_2021 ={response_gdp[1][2]['value']}')
# print(f'GDP_2020 ={response_gdp[1][3]['value']}')

In [ ]:
# link_pop = 'https://api.worldbank.org/v2/country/ABW/indicator/SP.POP.TOTL?date=2020:2023&format=json'
           
# response_pop = requests.get(link_pop).json()

# # pprint(response_pop)

# print(f'country ID ={response_pop[1][0]['country']['id']}')
# print(f'country ID ={response_pop[1][0]['country']['value']}')
# print(f'pop_2023 ={response_pop[1][0]['value']}')
# print(f'pop_2022 ={response_pop[1][1]['value']}')
# print(f'pop_2021 ={response_pop[1][2]['value']}')
# print(f'pop_2020 ={response_pop[1][3]['value']}')